In [6]:
import os
import glob
import fitz
from sentence_transformers import SentenceTransformer
from annoy import AnnoyIndex

c:\Users\sebas\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
# Ruta al directorio que contiene los archivos PDF
pdf_dir = "C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf"

# Ruta al modelo, por ejemplo, 'clip-ViT-L-14'
model_path = "clip-ViT-L-14"

# Cargar el modelo SentenceTransformer
model = SentenceTransformer(model_path)

# Función para extraer texto de los archivos PDF
def extraer_texto_de_pdf(pdf_path, max_length=77):
    texto = ""
    try:
        with fitz.open(pdf_path) as archivo_pdf:
            for num_pagina in range(len(archivo_pdf)):
                pagina = archivo_pdf[num_pagina]
                texto_pagina = pagina.get_text()
                # Truncar el texto de la página si es necesario
                if len(texto) + len(texto_pagina) > max_length:
                    texto_pagina = texto_pagina[:max_length - len(texto)]
                texto += texto_pagina
    except Exception as e:
        print(f"Error al leer el archivo PDF {pdf_path}: {e}")
    return texto


In [34]:
# Consulta de búsqueda
consulta = "ALQUILER DE VEHÍCULOS"

# Buscar archivos PDF
archivos = glob.glob(os.path.join(pdf_dir, "*.pdf"))

In [35]:
# Indexar archivos PDF
annoy_index = AnnoyIndex(768, metric='angular')
for idx, archivo in enumerate(archivos):
    texto = extraer_texto_de_pdf(archivo)
    incrustacion = model.encode([texto])[0]
    annoy_index.add_item(idx, incrustacion)
annoy_index.build(10)

True

In [36]:
# Codificar la consulta y encontrar archivos PDF similares
incrustacion_consulta = model.encode([consulta])[0]
indices_mas_cercanos, distancias_mas_cercanas = annoy_index.get_nns_by_vector(incrustacion_consulta, 5, include_distances=True)

In [37]:
# Mostrar o procesar los archivos PDF más cercanos
for idx, distancia in zip(indices_mas_cercanos, distancias_mas_cercanas):
    print(archivos[idx], distancia)

C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\Ejemplo1.pdf 0.7448084354400635
C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\Ejemplo2.pdf 0.7448084354400635
C:/Users/sebas/OneDrive/Documentos/Codigos Cemex/pdf\Ejemplo3.pdf 0.9151697754859924


In [33]:
print(embeddings.shape)


NameError: name 'embeddings' is not defined